In [91]:
import pandas as pd

title_df = pd.read_csv('data/title.basics.tsv', sep='\t', dtype=str, na_values=["\\N"])

crew_df = pd.read_csv("data/title.crew.tsv", sep="\t", dtype=str, na_values=["\\N"])

principals_df = pd.read_csv('data/title.principals.tsv', sep='\t', dtype=str, na_values=["\\N"])

ratings_df = pd.read_csv("data/title.ratings.tsv", sep="\t", dtype=str, na_values=["\\N"])

cast_names_df = pd.read_csv("data/name.basics.tsv", sep="\t", dtype=str, na_values=["\\N"])

director_names_df = pd.read_csv("data/name.basics.tsv", sep="\t", dtype=str, na_values=["\\N"])

In [92]:
title_df = title_df.loc[title_df['titleType'] == 'movie']
title_df = title_df.loc[title_df['isAdult'] == '0']
title_df.drop(columns=['titleType','originalTitle', 'isAdult', 'endYear'], inplace=True)

In [93]:
crew_df.drop(columns=['writers'], inplace=True)
crew_df["directors"] = crew_df["directors"].str.split(",")
crew_df = crew_df.explode("directors")[["tconst", "directors"]]

In [94]:
principals_df = principals_df.loc[(principals_df['category'] == 'actor') | (principals_df['category'] == 'actress')]
principals_df.drop(columns=['ordering', 'category', 'job', 'characters'], inplace=True)

In [95]:
cast_names_df.drop(columns=['birthYear', 'deathYear', 'primaryProfession', 'knownForTitles'], inplace=True)
director_names_df.drop(columns=['birthYear', 'deathYear', 'primaryProfession', 'knownForTitles'], inplace=True)

In [96]:
data = pd.merge(title_df, crew_df, on='tconst', how='left')

In [97]:
data = pd.merge(data, principals_df, on='tconst', how='left')

In [98]:
data = pd.merge(data, ratings_df, on='tconst', how='left')

In [99]:
data = pd.merge(data, cast_names_df, on='nconst', how='left')

In [ ]:
data = data.rename(columns={"primaryName" : "Cast"})
data.drop(columns=['nconst'], inplace=True)

In [101]:
data = pd.merge(data, director_names_df, left_on='directors', right_on='nconst', how='left')

In [ ]:
data = data.rename(columns={"primaryName" : "Director"})
data.drop(columns=['nconst', 'directors'], inplace=True)

In [103]:
data = data.fillna("Unknown")

In [104]:
data = data.groupby(['tconst']).agg({'primaryTitle' : 'first', 'startYear' : 'first', 'runtimeMinutes' : 'first', 'genres' : 'first', 'averageRating' : 'first', 'numVotes' : 'first', 'Cast': lambda x: ', '.join(x), 'Director': lambda x: ', '.join(set(x))}).reset_index()

In [ ]:
data = data.rename(columns={"tconst" : "ID", "primaryName" : "Director", "primaryTitle" : "Title", "startYear" : "Year", "runtimeMinutes" : "RunningTime", "genres": "Genres", "averageRating" : "Rating", "numVotes" : "Votes"})

In [113]:
data = data[['ID', 'Title', 'Year', 'Genres', 'Director', 'Cast', 'RunningTime', 'Rating', 'Votes']]

In [116]:
data.to_csv("data/data.csv", index=False, sep=";")